# БИБЛИОТЕКИ (install)

In [1]:
# 1. Удаляем старые версии MMCV, если есть
!pip uninstall -y mmcv mmcv-full
!pip install numpy==1.26.4

Found existing installation: mmcv 2.1.0
Uninstalling mmcv-2.1.0:
  Successfully uninstalled mmcv-2.1.0


In [2]:
!pip install torch==2.0.1+cu118 torchvision==0.15.2+cu118 \
    --extra-index-url https://download.pytorch.org/whl/cu118

# 2. Устанавливаем openmim (менеджер MMLab-пакетов), mmengine и совместимую версию mmcv
!pip install -U openmim
!mim install mmengine
!mim install "mmcv>=2.0.1,<2.2.0"

# 3. Устанавливаем MMPose (версия 1.x) и MMDetection
!mim install "mmpose>=1.1.0"
!mim install "mmdet>=3.1.0"

# 4. Устанавливаем DeepSort and OpenCV and YOLO
!pip install ultralytics deep-sort-realtime opencv-python
!git clone https://github.com/open-mmlab/mmpose.git
%cd mmpose
!pip install scipy tqdm mmpose==1.1.0 deep_sort_realtime --quiet
!pip install -r requirements.txt
!pip install -v -e .
%cd ..

import torch
from mmpose.apis import init_model, inference_topdown
from deep_sort_realtime.deepsort_tracker import DeepSort
print(torch.__version__)
print("Успешный импорт!")

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu118
Looking in links: https://download.openmmlab.com/mmcv/dist/cu118/torch2.0.0/index.html
Looking in links: https://download.openmmlab.com/mmcv/dist/cu118/torch2.0.0/index.html
  Using cached https://download.openmmlab.com/mmcv/dist/cu118/torch2.0.0/mmcv-2.1.0-cp311-cp311-manylinux1_x86_64.whl (98.6 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mmpose 1.3.2 requires mmdet<3.3.0,>=3.0.0; extra == "mim", but you have mmdet 3.3.0 which is incompatible.
Looking in links: https://download.openmmlab.com/mmcv/dist/cu118/torch2.0.0/index.html
  Cloning https://github.com/jin-s13/xtcocoapi to /tmp/pip-install-ijla_l36/xtcocotools_7a1cd2877ed64de8b61a401e491ff850
  Running command git clone --filter=blob:none --quiet https://github.com/jin-s13/xtcocoapi /tmp/pip-install-ijla_l36/xtcoco

Disabling PyTorch because PyTorch >= 2.1 is required but found 2.0.1+cu118


2.0.1+cu118
Успешный импорт!


In [3]:
import torch
from mmpose.apis import init_model, inference_topdown
from deep_sort_realtime.deepsort_tracker import DeepSort

print(torch.__version__)  # Должен быть 2.x
print("Успешный импорт!")


2.0.1+cu118
Успешный импорт!


# yolo11n-pose

In [ ]:

import time
import cv2
from ultralytics import YOLO
import os

# Путь к видео
youtube_video_path = "/content/input_video.mp4"
per_sent = 5
# Загрузка модели YOLO11-pose (nano-версия для скорости)
model = YOLO("yolo11n-pose.pt")

# Функция для получения длительности видео
def get_video_duration(video_path):
    if not os.path.exists(video_path):
        print(f"Error: Video file {video_path} not found")
        return 0, 0
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = cap.get(cv2.CAP_PROP_FRAME_COUNT)
    duration = total_frames / fps if fps > 0 else 0
    cap.release()
    return duration, fps

# Функция для обработки 10% видео
def process_video_segment(video_path, output_path, segment_percentage=per_sent / 100):
    # Проверка существования файла
    if not os.path.exists(video_path):
        print(f"Error: Video file {video_path} not found")
        return 0

    # Получение длительности и FPS
    total_duration, fps = get_video_duration(video_path)
    if total_duration == 0:
        return 0

    # Вычисление длительности сегмента (10%)
    segment_duration = total_duration * segment_percentage
    print(f"Total video duration: {total_duration:.2f} seconds")
    print(f"Processing segment duration: {segment_duration:.2f} seconds")

    # Открытие видео
    cap = cv2.VideoCapture(video_path)
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Настройка сохранения результата
    output_video_path = os.path.join(output_path, f"segment_output({per_sent}%).mp4")
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

    start_time = time.time()
    frame_count = 0
    max_frames = int(segment_duration * fps)

    while cap.isOpened() and frame_count < max_frames:
        ret, frame = cap.read()
        if not ret:
            break

        # Обработка кадра с помощью YOLO11-pose
        results = model.track(frame, persist=True, tracker="bytetrack.yaml")

        # Визуализация результатов
        annotated_frame = results[0].plot()  # Добавляет bounding boxes, ключевые точки и ID треков
        out.write(annotated_frame)

        frame_count += 1

    end_time = time.time()

    # Подсчет FPS
    inference_time = end_time - start_time
    processed_fps = frame_count / inference_time if inference_time > 0 else 0

    # Вывод результатов
    print(f"Processed {frame_count} frames")
    print(f"Inference time: {inference_time:.2f} seconds")
    print(f"Average FPS: {processed_fps:.2f}")

    # Извлечение ключевых точек и ID треков для первого кадра (пример)
    if results and len(results) > 0:
        print(f"Keypoints (xy): {results[0].keypoints.xy}")
        print(f"Track IDs: {results[0].boxes.id}")

    # Закрытие видео
    cap.release()
    out.release()
    print(f"Output saved to {output_video_path}")

    return processed_fps

# Создание директории для результатов
output_dir = "/content/output"
os.makedirs(output_dir, exist_ok=True)

# Обработка N% видео
print(f"Processing N% of video: {youtube_video_path}")
fps = process_video_segment(youtube_video_path, output_dir)


Processing N% of video: /content/input_video.mp4
Total video duration: 215.16 seconds
Processing segment duration: 10.76 seconds
requirements: Ultralytics requirement ['lap>=0.5.12'] not found, attempting AutoUpdate...

requirements: AutoUpdate success ✅ 0.6s
WARNING ⚠️ requirements: Restart runtime or rerun command for updates to take effect


0: 384x640 (no detections), 117.0ms
Speed: 2.1ms preprocess, 117.0ms inference, 2.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 13.4ms
Speed: 2.8ms preprocess, 13.4ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 11.4ms
Speed: 2.0ms preprocess, 11.4ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 9.6ms
Speed: 2.1ms preprocess, 9.6ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 10.6ms
Speed: 2.3ms preprocess, 10.6ms inference, 0.9ms postprocess per image at shape (1, 3,

# Faster R-CNN + DeepSort + HRNet

In [ ]:

# 2. Подключаем библиотеки и модели


import glob, scipy.io, torch, torchvision, cv2, numpy as np
from tqdm.notebook import tqdm
from deep_sort_realtime.deepsort_tracker import DeepSort
from mmpose.apis import init_model, inference_topdown

# --- Устройство ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# --- Детектор: Faster R-CNN ---
detector = torchvision.models.detection.fasterrcnn_resnet50_fpn(weights='DEFAULT').to(device)
detector.eval()

# --- Трекинг: DeepSort ---
tracker = DeepSort(max_age=5)

# --- Поза: HRNet ---
pose_config = 'mmpose/configs/body_2d_keypoint/topdown_heatmap/coco/td-hm_hrnet-w48_8xb32-210e_coco-256x192.py'
pose_checkpoint = 'hrnet_w48_coco_256x192.pth'
if not os.path.exists(pose_checkpoint):
    !wget -q https://download.openmmlab.com/mmpose/top_down/hrnet/hrnet_w48_coco_256x192-b9e0b3ab_20200708.pth -O hrnet_w48_coco_256x192.pth
pose_model = init_model(pose_config, pose_checkpoint, device=device)
#___| don't change |__

Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth
100%|██████████| 160M/160M [00:00<00:00, 235MB/s]


Loads checkpoint by local backend from path: hrnet_w48_coco_256x192.pth


/content/mmpose/mmpose/datasets/datasets/utils.py:102: UserWarning: The metainfo config file "configs/_base_/datasets/coco.py" does not exist. A matched config file "/content/mmpose/mmpose/.mim/configs/_base_/datasets/coco.py" will be used instead.
  warnings.warn(


In [ ]:
import torch, torchvision
import os, cv2, time
import numpy as np
from tqdm.notebook import tqdm
from deep_sort_realtime.deepsort_tracker import DeepSort
from mmpose.apis import init_model, inference_topdown

# --- Устройство ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# --- Детектор: Faster R-CNN ---
detector = torchvision.models.detection.fasterrcnn_resnet50_fpn(weights='DEFAULT').to(device)
detector.eval()

# --- Трекинг: DeepSort ---
tracker = DeepSort(max_age=5,n_init=1)

# --- Поза: HRNet ---
pose_config = 'mmpose/configs/body_2d_keypoint/topdown_heatmap/coco/td-hm_hrnet-w48_8xb32-210e_coco-256x192.py'
pose_checkpoint = 'hrnet_w48_coco_256x192.pth'
if not os.path.exists(pose_checkpoint):
    !wget -q https://download.openmmlab.com/mmpose/top_down/hrnet/hrnet_w48_coco_256x192-b9e0b3ab_20200708.pth -O hrnet_w48_coco_256x192.pth
pose_model = init_model(pose_config, pose_checkpoint, device=device)


Loads checkpoint by local backend from path: hrnet_w48_coco_256x192.pth


In [ ]:

N = 0.05
# --- Путь к видео ---
video_path = '/content/input_video.mp4'
output_path =f'/content/output_video({N}%).mp4'
assert os.path.exists(video_path), f"Файл {video_path} не найден"

# --- Открываем видео ---
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    raise IOError(f"Не удалось открыть видео {video_path}")

fps = cap.get(cv2.CAP_PROP_FPS)
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# --- Обрезка N% из центра видео ---
  # процент видео (например, 2%)
segment_frames = max(1, int(total_frames * N / 100))

start_frame = (total_frames - segment_frames) // 2
end_frame = start_frame + segment_frames
cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)

# --- Видеозапись выходного результата ---
out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (w, h))

# --- Визуализация скелета (COCO) ---
skeleton = [[0,1],[0,2],[1,3],[2,4],[0,5],[0,6],
            [5,7],[7,9],[6,8],[8,10],[5,11],[6,12],
            [11,13],[13,15],[12,14],[14,16]]

predictions = {}
saved_frames = 0

# --- Старт обработки ---
print(f"▶️ Начинаем обработку {segment_frames} кадров из центра видео ({start_frame} → {end_frame})")
start_time = time.time()

with tqdm(total=segment_frames, desc=f'Processing {N}% center of video') as pbar:
    for frame_idx in range(start_frame, end_frame):
        ret, frame = cap.read()
        if not ret:
            print(f"[!] Видео закончилось на кадре {frame_idx}")
            break

        # --- Детекция людей ---
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        input_tensor = torchvision.transforms.ToTensor()(frame_rgb).to(device)
        with torch.no_grad():
            outputs = detector([input_tensor])[0]

        boxes = outputs['boxes'].cpu().numpy()
        scores = outputs['scores'].cpu().numpy()
        labels = outputs['labels'].cpu().numpy()
        mask = (scores > 0.5) & (labels == 1)
        boxes = boxes[mask]; scores = scores[mask]

        # --- Трекинг ---
        detections = []
        for (x1,y1,x2,y2), score in zip(boxes, scores):
            w_box, h_box = x2 - x1, y2 - y1
            detections.append(([int(x1), int(y1), int(w_box), int(h_box)], float(score), 'person'))
        tracks = tracker.update_tracks(detections, frame=frame)

        frame_results = []
        for track in tracks:
            if not track.is_confirmed():
                continue
            track_id = track.track_id
            l,t,r,b = track.to_ltrb(orig=True)
            frame_results.append({'track_id': track_id, 'bbox': [l,t,r,b]})

        # --- Оценка позы ---
        for res in frame_results:
            l,t,r,b = res['bbox']
            pose_res = inference_topdown(pose_model, frame, bboxes=np.array([[l,t,r,b]], dtype=np.float32))
            pred_instances = pose_res[0].pred_instances
            if pred_instances.keypoints is not None and len(pred_instances.keypoints) > 0:
                res['keypoints'] = pred_instances.keypoints[0]
                res['keypoint_scores'] = pred_instances.keypoint_scores[0]

        predictions[frame_idx] = frame_results

        # --- Визуализация ---
        for res in frame_results:
            l,t,r,b = res['bbox']
            cv2.rectangle(frame, (int(l),int(t)), (int(r),int(b)), (0,255,0), 2)
            cv2.putText(frame, f'ID {res["track_id"]}', (int(l), int(t)-10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,0), 2)
            if 'keypoints' in res:
                kps = res['keypoints']
                ks = res['keypoint_scores']
                for idx, (x, y) in enumerate(kps):
                    if ks[idx] > 0.3:
                        cv2.circle(frame, (int(x), int(y)), 3, (0,0,255), -1)
                for (p, q) in skeleton:
                    if ks[p] > 0.3 and ks[q] > 0.3:
                        pt1 = (int(kps[p][0]), int(kps[p][1]))
                        pt2 = (int(kps[q][0]), int(kps[q][1]))
                        cv2.line(frame, pt1, pt2, (255, 0, 0), 2)

        out.write(frame)
        saved_frames += 1
        pbar.update(1)

# --- Завершаем ---
cap.release()
out.release()
end_time = time.time()
duration = end_time - start_time
fps_processed = saved_frames / duration if duration > 0 else 0

print(f"\n✅ Обработка завершена: {saved_frames} кадров за {duration:.1f} сек. ({fps_processed:.2f} FPS)")
print(f"📦 Выходной файл: {output_path}")
!ls -lh {output_path}


▶️ Начинаем обработку 2 кадров из центра видео (2688 → 2690)


Processing 0.05% center of video:   0%|          | 0/2 [00:00<?, ?it/s]


✅ Обработка завершена: 2 кадров за 2.2 сек. (0.93 FPS)
📦 Выходной файл: /content/output_video(0.05%).mp4
/bin/bash: -c: line 1: syntax error near unexpected token `('
/bin/bash: -c: line 1: `ls -lh /content/output_video(0.05%).mp4'


# Penn_Action

In [ ]:
# 1. Скачивание и распаковка
import os, urllib.request, tarfile
DATA_ROOT = '/content/Penn_Action'

if not os.path.exists(os.path.join(DATA_ROOT, 'frames')):
    print("⬇️ Скачиваем Penn Action...")
    urllib.request.urlretrieve(
        'https://www.cis.upenn.edu/~kostas/Penn_Action.tar.gz',
        '/content/Penn_Action.tar.gz'
    )
    print("🗂️ Распаковываем...")
    with tarfile.open('/content/Penn_Action.tar.gz', 'r:gz') as f:
        f.extractall('/content/')
    os.rename('/content/Penn_Action', DATA_ROOT)
print("✅ Датасет готов:", os.listdir(DATA_ROOT))


⬇️ Скачиваем Penn Action...
🗂️ Распаковываем...
✅ Датасет готов: ['frames', 'labels', 'tools', 'README']


In [ ]:
import scipy.io as sio
mat = sio.loadmat('/content/Penn_Action/labels/0001.mat')
print(mat['x'].shape, mat['y'].shape, mat['visibility'].shape)  # (nframes, 13)
print(["head","l_sh","r_sh","l_el","r_el","l_wr","r_wr","l_hi","r_hi","l_kn","r_kn","l_an","r_an"])


(151, 13) (151, 13) (151, 13)
['head', 'l_sh', 'r_sh', 'l_el', 'r_el', 'l_wr', 'r_wr', 'l_hi', 'r_hi', 'l_kn', 'r_kn', 'l_an', 'r_an']


In [ ]:
import os, cv2, glob, numpy as np, torch, torchvision
from tqdm.notebook import tqdm
from deep_sort_realtime.deepsort_tracker import DeepSort
from mmpose.apis import init_model, inference_topdown

# === Параметры ===
DATA_ROOT = '/content/Penn_Action'
FRAMES_DIR = os.path.join(DATA_ROOT, 'frames')
OUTPUT_VIDEO = '/content/output_annotated_unlabeled_ID.mp4'
V = 1  # количество людей для отображения

# Инициализация моделей и трекера
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
det = torchvision.models.detection.fasterrcnn_resnet50_fpn(weights='DEFAULT').to(device)
det.eval()
tracker = DeepSort(max_age=5, n_init=1)

pose_cfg = 'mmpose/configs/body_2d_keypoint/topdown_heatmap/coco/td-hm_hrnet-w48_8xb32-210e_coco-256x192.py'
pose_ckpt = 'hrnet_w48_coco_256x192.pth'
if not os.path.exists(pose_ckpt):
    !wget -q https://download.openmmlab.com/mmpose/top_down/hrnet/hrnet_w48_coco_256x192-b9e0b3ab_20200708.pth -O {pose_ckpt}
pose_model = init_model(pose_cfg, pose_ckpt, device=device)

# Выбор первого видео с ≥20 кадров
vids = sorted(os.listdir(FRAMES_DIR))
for vid in vids:
    files = sorted(glob.glob(os.path.join(FRAMES_DIR, vid, '*.jpg')))
    if len(files) >= 20:
        pick_vid = vid
        break

frames = sorted(glob.glob(os.path.join(FRAMES_DIR, pick_vid, '*.jpg')))
total = len(frames)
seg =150
start = max(0, (total - seg)//2)
selected = frames[start:start+seg]

# Настройка видео write
sample = cv2.imread(selected[0])
h, w = sample.shape[:2]
out = cv2.VideoWriter(OUTPUT_VIDEO, cv2.VideoWriter_fourcc(*'mp4v'), 15, (w,h))

# Скелет COCO
skeleton = [[0,1],[0,2],[1,3],[2,4],[0,5],[0,6],
            [5,7],[7,9],[6,8],[8,10],[5,11],[6,12],
            [11,13],[13,15],[12,14],[14,16]]

print(f"▶️ Обрабатываем '{pick_vid}', кадры {start}-{start+seg-1}")

for fp in tqdm(selected):
    frame = cv2.imread(fp)
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    t = torchvision.transforms.ToTensor()(rgb).to(device)

    with torch.no_grad():
        out_det = det([t])[0]

    mask = (out_det['scores']>0.5)&(out_det['labels']==1)
    boxes = out_det['boxes'][mask].cpu().numpy()
    scores = out_det['scores'][mask].cpu().numpy()
    dets = [([int(b[0]),int(b[1]),int(b[2]-b[0]),int(b[3]-b[1])], s, 'person')
            for b,s in zip(boxes, scores)]
    tracks = tracker.update_tracks(dets, frame=frame)

    # Собираем подтверждённые треки и сортируем по track_id
    confirmed = [tr for tr in tracks if tr.is_confirmed()]
    confirmed.sort(key=lambda tr: tr.track_id)

    # Выбираем первые V по ID
    for tr in confirmed[:V]:
        tid = tr.track_id
        l,t_,u,v = map(int, tr.to_ltrb(orig=True))
        cv2.rectangle(frame, (l,t_), (u,v), (0,255,0), 2)
        cv2.putText(frame, f'ID {tid}', (l, t_-10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,0), 2)

        pose = inference_topdown(pose_model, frame, np.array([[l,t_,u,v]], dtype=np.float32))[0]
        if pose.pred_instances.keypoints is not None:
            kps = pose.pred_instances.keypoints[0]
            ksc = pose.pred_instances.keypoint_scores[0]
            for i,(x,y) in enumerate(kps):
                if ksc[i] > 0.3:
                    cv2.circle(frame, (int(x),int(y)), 3, (0,0,255), -1)
            for p,q in skeleton:
                if ksc[p]>0.3 and ksc[q]>0.3:
                    pt1 = tuple(kps[p].astype(int))
                    pt2 = tuple(kps[q].astype(int))
                    cv2.line(frame, pt1, pt2, (255,0,0), 2)

    out.write(frame)

out.release()
print(f"✅ Сохранено: {OUTPUT_VIDEO}")


Loads checkpoint by local backend from path: hrnet_w48_coco_256x192.pth
▶️ Обрабатываем '0001', кадры 0-149


/content/mmpose/mmpose/datasets/datasets/utils.py:102: UserWarning: The metainfo config file "configs/_base_/datasets/coco.py" does not exist. A matched config file "/content/mmpose/mmpose/.mim/configs/_base_/datasets/coco.py" will be used instead.
  warnings.warn(


  0%|          | 0/150 [00:00<?, ?it/s]

✅ Сохранено: /content/output_annotated_unlabeled_ID.mp4


In [ ]:
import os, cv2, glob, numpy as np, torch, torchvision, scipy.io
from tqdm import tqdm
from deep_sort_realtime.deepsort_tracker import DeepSort
from mmpose.apis import init_model, inference_topdown

# --- Параметры ---
DATA_ROOT = '/content/Penn_Action'
FRAMES_DIR = os.path.join(DATA_ROOT, 'frames')
LABELS_DIR = os.path.join(DATA_ROOT, 'labels')
POSE_CFG = 'mmpose/configs/body_2d_keypoint/topdown_heatmap/coco/td-hm_hrnet-w48_8xb32-210e_coco-256x192.py'
POSE_CKPT = 'hrnet_w48_coco_256x192.pth'
TARGET_FRAMES = 100  # нужно собрать минимум столько кадров
MAX_FRAMES_PER_VIDEO = 20

# --- Инициализация моделей ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
detector = torchvision.models.detection.fasterrcnn_resnet50_fpn(weights='DEFAULT').to(device)
detector.eval()
tracker = DeepSort(max_age=5, n_init=1)
if not os.path.exists(POSE_CKPT):
    os.system(f'wget -q https://download.openmmlab.com/mmpose/top_down/hrnet/hrnet_w48_coco_256x192-b9e0b3ab_20200708.pth -O {POSE_CKPT}')
pose_model = init_model(POSE_CFG, POSE_CKPT, device=device)

# --- Сбор видео с наибольшей длиной анотаций ---
video_lengths = []
for vid in os.listdir(FRAMES_DIR):
    files = glob.glob(f'{FRAMES_DIR}/{vid}/*.jpg')
    if os.path.exists(f'{LABELS_DIR}/{vid}.mat'):
        mat = scipy.io.loadmat(f'{LABELS_DIR}/{vid}.mat')
        nframes = int(mat['nframes'][0][0])
        video_lengths.append((vid, nframes))

video_lengths.sort(key=lambda x: -x[1])  # от большего к меньшему

frames_list, collected, processed_videos = [], 0, []

for vid, vlen in video_lengths:
    if collected >= TARGET_FRAMES:
        break

    frame_files = sorted(glob.glob(f'{FRAMES_DIR}/{vid}/*.jpg'))
    mat = scipy.io.loadmat(f'{LABELS_DIR}/{vid}.mat')
    available = min(MAX_FRAMES_PER_VIDEO, vlen, len(frame_files), TARGET_FRAMES - collected)

    if available < 5:
        continue

    for idx in range(available):
        frames_list.append((frame_files[idx], mat, idx, vid))

    collected += available
    processed_videos.append((vid, available))

print(f"Кадров для обработки: {len(frames_list)}")
print("Обработанные видео:")
for vid, count in processed_videos:
    print(f" - {vid}: {count} кадров")

# --- Метрики ---
all_pred_kps, all_true_kps, all_visible = [], [], []
all_pred_bboxes, all_true_bboxes = [], []

# --- Скелет COCO ---
skeleton = [[0,1],[0,2],[1,3],[2,4],[0,5],[0,6],[5,7],[7,9],[6,8],[8,10],[5,11],[6,12],[11,13],[13,15],[12,14],[14,16]]

# --- Обработка кадров ---
for path, mat, idx, vid in tqdm(frames_list):
    frame = cv2.imread(path)
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    t = torchvision.transforms.ToTensor()(rgb).to(device)

    with torch.no_grad():
        out_det = detector([t])[0]

    mask = (out_det['scores'] > 0.5) & (out_det['labels'] == 1)
    boxes = out_det['boxes'][mask].cpu().numpy()
    scores = out_det['scores'][mask].cpu().numpy()
    dets = [([int(b[0]), int(b[1]), int(b[2]-b[0]), int(b[3]-b[1])], s, 'person') for b, s in zip(boxes, scores)]
    tracks = tracker.update_tracks(dets, frame=frame)

    confirmed = [tr for tr in tracks if tr.is_confirmed()]
    confirmed.sort(key=lambda tr: tr.track_id)

    if len(confirmed) == 0:
        continue

    tr = confirmed[0]  # используем только первого человека
    l, t_, u, v = map(int, tr.to_ltrb(orig=True))
    bbox_pred = [l, t_, u, v]
    out_pose = inference_topdown(pose_model, frame, np.array([[l, t_, u, v]], dtype=np.float32))[0]

    if out_pose.pred_instances.keypoints is None:
        continue

    kps_pred = out_pose.pred_instances.keypoints[0]
    ksc = out_pose.pred_instances.keypoint_scores[0]

    # GT keypoints Penn (13 → COCO 17)
    x_gt, y_gt, vis_gt = mat['x'][idx], mat['y'][idx], mat['visibility'][idx].astype(bool)
    true_kps_coco, vis_coco = np.full((17, 2), np.nan), np.zeros(17, dtype=bool)
    map_penn_to_coco = {0:0, 1:5, 2:6, 3:7, 4:8, 5:9, 6:10, 7:11, 8:12, 9:13, 10:14, 11:15, 12:16}

    for p_idx, coco_idx in map_penn_to_coco.items():
        true_kps_coco[coco_idx] = [x_gt[p_idx], y_gt[p_idx]]
        vis_coco[coco_idx] = vis_gt[p_idx]

    all_pred_kps.append(kps_pred)
    all_true_kps.append(true_kps_coco)
    all_visible.append(vis_coco)

    bbox_gt = mat['bbox'][idx]
    all_true_bboxes.append(bbox_gt)
    all_pred_bboxes.append(bbox_pred)

# --- Расчёт метрик ---
visible = np.stack(all_visible)
true_kps, pred_kps = np.stack(all_true_kps), np.stack(all_pred_kps)
valid = visible & ~np.isnan(true_kps[:, :, 0])
errors = np.linalg.norm(pred_kps - true_kps, axis=2)
mae, mse = np.mean(errors[valid]), np.mean(errors[valid]**2)

def iou(boxA, boxB):
    xA, yA = max(boxA[0], boxB[0]), max(boxA[1], boxB[1])
    xB, yB = min(boxA[2], boxB[2]), min(boxA[3], boxB[3])
    inter = max(0, xB - xA) * max(0, yB - yA)
    areaA = max(0, boxA[2] - boxA[0]) * max(0, boxA[3] - boxA[1])
    areaB = max(0, boxB[2] - boxB[0]) * max(0, boxB[3] - boxB[1])
    union = areaA + areaB - inter
    return inter / union if union > 0 else 0

iou_scores = [iou(pred, true) for pred, true in zip(all_pred_bboxes, all_true_bboxes)]

print("\n📊 Метрики:")
print(f"MAE по keypoints: {mae:.2f} пикселя")
print(f"MSE по keypoints: {mse:.2f} пикселя²")
print(f"Средний IOU по bbox: {np.mean(iou_scores):.3f}")


Loads checkpoint by local backend from path: hrnet_w48_coco_256x192.pth
Кадров для обработки: 100
Обработанные видео:
 - 0758: 20 кадров
 - 0748: 20 кадров
 - 0623: 20 кадров
 - 0731: 20 кадров
 - 0755: 20 кадров


100%|██████████| 100/100 [00:23<00:00,  4.17it/s]


📊 Метрики:
MAE по keypoints: 56.94 пикселя
MSE по keypoints: 4617.94 пикселя²
Средний IOU по bbox: 0.714


In [ ]:
import os
import cv2
import glob
import numpy as np
import torch
import torchvision
import scipy.io
from tqdm import tqdm
from deep_sort_realtime.deepsort_tracker import DeepSort
from mmpose.apis import init_model, inference_topdown

# --- Параметры ---
DATA_ROOT = '/content/Penn_Action'
FRAMES_DIR = os.path.join(DATA_ROOT, 'frames')
LABELS_DIR = os.path.join(DATA_ROOT, 'labels')
POSE_CFG = 'mmpose/configs/body_2d_keypoint/topdown_heatmap/coco/td-hm_hrnet-w48_8xb32-210e_coco-256x192.py'
POSE_CKPT = 'hrnet_w48_coco_256x192.pth'
TARGET_FRAMES = 1000  # нужно собрать минимум столько кадров
MAX_FRAMES_PER_VIDEO = 10

# --- Инициализация моделей ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
detector = torchvision.models.detection.fasterrcnn_resnet50_fpn(weights='DEFAULT').to(device)
detector.eval()
tracker = DeepSort(max_age=5, n_init=1)
if not os.path.exists(POSE_CKPT):
    os.system(f'wget -q https://download.openmmlab.com/mmpose/top_down/hrnet/hrnet_w48_coco_256x192-b9e0b3ab_20200708.pth -O {POSE_CKPT}')
pose_model = init_model(POSE_CFG, POSE_CKPT, device=device)

# --- Сбор видео с наибольшей длиной анотаций ---
video_lengths = []
for vid in os.listdir(FRAMES_DIR):
    files = glob.glob(f'{FRAMES_DIR}/{vid}/*.jpg')
    if os.path.exists(f'{LABELS_DIR}/{vid}.mat'):
        mat = scipy.io.loadmat(f'{LABELS_DIR}/{vid}.mat')
        nframes = int(mat['nframes'][0][0])
        video_lengths.append((vid, nframes))

video_lengths.sort(key=lambda x: -x[1])  # от большего к меньшему

frames_list, collected, processed_videos = [], 0, []

for vid, vlen in video_lengths:
    if collected >= TARGET_FRAMES:
        break

    frame_files = sorted(glob.glob(f'{FRAMES_DIR}/{vid}/*.jpg'))
    mat = scipy.io.loadmat(f'{LABELS_DIR}/{vid}.mat')
    available = min(MAX_FRAMES_PER_VIDEO, vlen, len(frame_files), TARGET_FRAMES - collected)

    if available < 5:
        continue

    for idx in range(available):
        frames_list.append((frame_files[idx], mat, idx, vid))

    collected += available
    processed_videos.append((vid, available))

print(f"Кадров для обработки: {len(frames_list)}")
print("Обработанные видео:")
for vid, count in processed_videos:
    print(f" - {vid}: {count} кадров")

# --- Метрики ---
all_pred_kps, all_true_kps, all_visible = [], [], []
all_pred_bboxes, all_true_bboxes = [], []

# --- Скелет COCO ---
skeleton = [[0,1],[0,2],[1,3],[2,4],[0,5],[0,6],[5,7],[7,9],[6,8],[8,10],[5,11],[6,12],[11,13],[13,15],[12,14],[14,16]]

# --- Функция подсчёта IOU ---
def iou(boxA, boxB):
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    interW = max(0, xB - xA)
    interH = max(0, yB - yA)
    interArea = interW * interH
    boxAArea = max(0, boxA[2] - boxA[0]) * max(0, boxA[3] - boxA[1])
    boxBArea = max(0, boxB[2] - boxB[0]) * max(0, boxB[3] - boxB[1])
    union = boxAArea + boxBArea - interArea
    if union == 0:
        return 0
    return interArea / union

# --- Обработка кадров ---
for path, mat, idx, vid in tqdm(frames_list):
    frame = cv2.imread(path)
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    t = torchvision.transforms.ToTensor()(rgb).to(device)

    with torch.no_grad():
        out_det = detector([t])[0]

    mask = (out_det['scores'] > 0.5) & (out_det['labels'] == 1)
    boxes = out_det['boxes'][mask].cpu().numpy()
    scores = out_det['scores'][mask].cpu().numpy()
    dets = [([int(b[0]), int(b[1]), int(b[2]-b[0]), int(b[3]-b[1])], s, 'person') for b, s in zip(boxes, scores)]
    tracks = tracker.update_tracks(dets, frame=frame)

    confirmed = [tr for tr in tracks if tr.is_confirmed()]
    confirmed.sort(key=lambda tr: tr.track_id)

    if len(confirmed) == 0:
        continue

    tr = confirmed[0]  # используем только первого человека
    l, t_, u, v = map(int, tr.to_ltrb(orig=True))
    bbox_pred = [l, t_, u, v]

    out_pose = inference_topdown(pose_model, frame, np.array([[l, t_, u, v]], dtype=np.float32))[0]

    if out_pose.pred_instances.keypoints is None:
        continue

    kps_pred = out_pose.pred_instances.keypoints[0]
    ksc = out_pose.pred_instances.keypoint_scores[0]

    # GT keypoints Penn (13 → COCO 17)
    x_gt, y_gt, vis_gt = mat['x'][idx], mat['y'][idx], mat['visibility'][idx].astype(bool)
    true_kps_coco = np.full((17, 2), np.nan)
    vis_coco = np.zeros(17, dtype=bool)
    map_penn_to_coco = {0:0, 1:5, 2:6, 3:7, 4:8, 5:9, 6:10, 7:11, 8:12, 9:13, 10:14, 11:15, 12:16}

    for p_idx, coco_idx in map_penn_to_coco.items():
        true_kps_coco[coco_idx] = [x_gt[p_idx], y_gt[p_idx]]
        vis_coco[coco_idx] = vis_gt[p_idx]

    all_pred_kps.append(kps_pred)
    all_true_kps.append(true_kps_coco)
    all_visible.append(vis_coco)

    bbox_gt = mat['bbox'][idx]
    all_true_bboxes.append(bbox_gt)
    all_pred_bboxes.append(bbox_pred)

# --- Расчёт метрик ---
visible = np.stack(all_visible)
true_kps = np.stack(all_true_kps)
pred_kps = np.stack(all_pred_kps)
valid = visible & ~np.isnan(true_kps[:, :, 0])
errors = np.linalg.norm(pred_kps - true_kps, axis=2)
mae = np.mean(errors[valid])
mse = np.mean(errors[valid]**2)

iou_scores = [iou(pred, true) for pred, true in zip(all_pred_bboxes, all_true_bboxes)]
mean_iou = np.mean(iou_scores)

print("\n📊 Метрики в формате Penn:")
print(f"MAE по keypoints: {mae:.2f} пикселя")
print(f"MSE по keypoints: {mse:.2f} пикселя²")
print(f"Средний IOU по bbox: {mean_iou:.3f}")


Loads checkpoint by local backend from path: hrnet_w48_coco_256x192.pth
Кадров для обработки: 1000
Обработанные видео:
 - 0758: 10 кадров
 - 0748: 10 кадров
 - 0623: 10 кадров
 - 0731: 10 кадров
 - 0755: 10 кадров
 - 0736: 10 кадров
 - 0750: 10 кадров
 - 0723: 10 кадров
 - 0761: 10 кадров
 - 0746: 10 кадров
 - 0737: 10 кадров
 - 0762: 10 кадров
 - 0719: 10 кадров
 - 0726: 10 кадров
 - 0753: 10 кадров
 - 0779: 10 кадров
 - 0754: 10 кадров
 - 0715: 10 кадров
 - 0735: 10 кадров
 - 0787: 10 кадров
 - 0744: 10 кадров
 - 0781: 10 кадров
 - 0776: 10 кадров
 - 0752: 10 кадров
 - 0740: 10 кадров
 - 0774: 10 кадров
 - 0767: 10 кадров
 - 0702: 10 кадров
 - 0785: 10 кадров
 - 0778: 10 кадров
 - 0727: 10 кадров
 - 0769: 10 кадров
 - 0741: 10 кадров
 - 0716: 10 кадров
 - 1173: 10 кадров
 - 0770: 10 кадров
 - 0768: 10 кадров
 - 0734: 10 кадров
 - 0708: 10 кадров
 - 0751: 10 кадров
 - 0728: 10 кадров
 - 0718: 10 кадров
 - 0738: 10 кадров
 - 0773: 10 кадров
 - 0763: 10 кадров
 - 0729: 10 кадров
 - 0749

100%|██████████| 1000/1000 [03:58<00:00,  4.20it/s]


📊 Метрики в формате Penn:
MAE по keypoints: 65.45 пикселя
MSE по keypoints: 8267.59 пикселя²
Средний IOU по bbox: 0.530


In [ ]:
import scipy.io
video_id = '0001'
labels_dir = '/content/Penn_Action/labels'
label_path = os.path.join(labels_dir, f'{video_id}.mat')

mat = scipy.io.loadmat(label_path)
# Проверяем содержимое
print(mat.keys())


dict_keys(['__header__', '__version__', '__globals__', 'action', 'pose', 'x', 'y', 'visibility', 'train', 'bbox', 'dimensions', 'nframes'])


In [ ]:
import os
import cv2
import numpy as np
import scipy.io

# Путь к датасету
DATASET_DIR = '/content/Penn_Action'
video_id = '0010'

frames_dir = os.path.join(DATASET_DIR, 'frames', video_id)
label_path = os.path.join(DATASET_DIR, 'labels', f'{video_id}.mat')

# Загружаем аннотацию
mat = scipy.io.loadmat(label_path)

# Извлекаем поля напрямую
xs = mat['x']            # shape: (n_frames, 13)
ys = mat['y']
vis = mat['visibility']  # shape: (n_frames, 13)
bboxes = mat['bbox']     # shape: (n_frames, 4)

nframes = int(mat['nframes'][0][0])

# Получим список кадров
frame_files = sorted(os.listdir(frames_dir))
frame_paths = [os.path.join(frames_dir, f) for f in frame_files if f.endswith('.jpg')]

# Настройка видео-сохранения
sample_frame = cv2.imread(frame_paths[0])
height, width = sample_frame.shape[:2]

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out_path = f'/content/penn_action_annotated_{video_id}.mp4'
out = cv2.VideoWriter(out_path, fourcc, 15, (width, height))

# Цвета и связи (скелет)
color = (0, 255, 0)
joint_pairs = [
    (0,1), (1,3), (0,2), (2,4),  # голова → плечи → локти
    (1,5), (2,6),                # плечи → бедра
    (5,7), (6,8),                # бедра → колени
    (7,9), (8,10),              # колени → лодыжки
    (1,2)                        # между плечами
]

# Визуализация
for i in range(min(nframes, len(frame_paths))):
    frame = cv2.imread(frame_paths[i])
    if frame is None:
        continue

    # Нарисуем ключевые точки
    for j in range(13):
        if vis[i, j]:
            x, y = int(xs[i, j]), int(ys[i, j])
            cv2.circle(frame, (x, y), 4, color, -1)

    # Нарисуем соединения
    for j1, j2 in joint_pairs:
        if vis[i, j1] and vis[i, j2]:
            pt1 = (int(xs[i, j1]), int(ys[i, j1]))
            pt2 = (int(xs[i, j2]), int(ys[i, j2]))
            cv2.line(frame, pt1, pt2, (255, 0, 0), 2)

    # Нарисуем bbox
    x1, y1, x2, y2 = map(int, bboxes[i])
    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)

    out.write(frame)

out.release()
print(f"✅ Сохранено видео с аннотацией: {out_path}")


✅ Сохранено видео с аннотацией: /content/penn_action_annotated_0010.mp4


#  Сравнение yolo11n-pose и  Faster R-CNN + HRNet

In [9]:
import cv2
import time
import numpy as np
from ultralytics import YOLO
from tqdm import tqdm
import torch
import torchvision
from mmpose.apis import init_model, inference_topdown
from deep_sort_realtime.deepsort_tracker import DeepSort
import os
import time
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Устройство для вычислений
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Параметры
video_path = '/content/input_video.mp4'
output_dir = '/content/output'
os.makedirs(output_dir, exist_ok=True)
yolo_output_path = os.path.join(output_dir, 'output_yolo.mp4')
second_output_path = os.path.join(output_dir, 'output_second.mp4')

# Создание выходных видео
cap = cv2.VideoCapture(video_path)
fps = cap.get(cv2.CAP_PROP_FPS)
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

out_yolo = cv2.VideoWriter(yolo_output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (w, h))
out_second = cv2.VideoWriter(second_output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (w, h))

POSE_CFG = 'mmpose/configs/body_2d_keypoint/topdown_heatmap/coco/td-hm_hrnet-w48_8xb32-210e_coco-256x192.py'
POSE_CKPT = 'hrnet_w48_coco_256x192.pth'

# Загрузка модели YOLO
yolo_model = YOLO("yolo11n-pose.pt")

# Инициализация моделей
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
detector = torchvision.models.detection.fasterrcnn_resnet50_fpn(weights='DEFAULT').to(device)
detector.eval()
tracker = DeepSort(max_age=25, n_init=3)

# Загрузка модели pose
if not os.path.exists(POSE_CKPT):
    os.system(f'wget -q https://download.openmmlab.com/mmpose/top_down/hrnet/hrnet_w48_coco_256x192-b9e0b3ab_20200708.pth -O {POSE_CKPT}')
pose_model = init_model(POSE_CFG, POSE_CKPT, device=device)

# Скелет для визуализации
skeleton = [[0,1],[0,2],[1,3],[2,4],[0,5],[0,6],
            [5,7],[7,9],[6,8],[8,10],[5,11],[6,12],
            [11,13],[13,15],[12,14],[14,16]]

# Функция для вычисления IOU между двумя боксами
def calculate_iou(box1, box2):
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])
    intersection = max(0, x2 - x1) * max(0, y2 - y1)
    area1 = (box1[2] - box1[0]) * (box1[3] - box1[1])
    area2 = (box2[2] - box2[0]) * (box2[3] - box2[1])
    union = area1 + area2 - intersection
    return intersection / union if union > 0 else 0

# Открытие видео
cap = cv2.VideoCapture(video_path)
fps = cap.get(cv2.CAP_PROP_FPS)
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Определение сегмента: 5% кадров из центра
N = 20  # процент
segment_frames = max(1, int(total_frames * N / 100))
start_frame = (total_frames - segment_frames) // 2
end_frame = start_frame + segment_frames
cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)

# Списки для хранения метрик
iou_list = []
mae_list = []
mse_list = []

# Для отслеживания времени и скорости обработки
yolo_times = []
second_model_times = []

# Обработка каждого кадра
print(f"▶️ Обработка видео: {os.path.basename(video_path)}, кадры {start_frame}-{end_frame-1}")
# Время обработки каждого кадра
for _ in tqdm(range(segment_frames)):  # Упрощенный прогресс-бар
    ret, frame = cap.read()
    if not ret:
        break

    # Время обработки кадра для YOLO
    start_time = time.time()
    # Обработка моделью YOLO (считается "истиной")
    yolo_results = yolo_model.track(frame, persist=True, tracker="bytetrack.yaml")
    yolo_boxes = yolo_results[0].boxes.xyxy.cpu().numpy()
    yolo_keypoints = yolo_results[0].keypoints.xy.cpu().numpy()
    yolo_time = time.time() - start_time
    yolo_times.append(yolo_time)

    # Время обработки кадра для второй модели (Faster R-CNN + HRNet)
    start_time = time.time()
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    input_tensor = torchvision.transforms.ToTensor()(frame_rgb).to(device)
    with torch.no_grad():
        outputs = detector([input_tensor])[0]
    boxes = outputs['boxes'].cpu().numpy()
    scores = outputs['scores'].cpu().numpy()
    labels = outputs['labels'].cpu().numpy()
    mask = (scores > 0.5) & (labels == 1)  # Фильтр по уверенности и классу "человек"
    boxes = boxes[mask]

    detections = []
    for (x1, y1, x2, y2) in boxes:
        w_box, h_box = x2 - x1, y2 - y1
        detections.append(([int(x1), int(y1), int(w_box), int(h_box)], 1.0, 'person'))

    tracks = tracker.update_tracks(detections, frame=frame)
    second_results = []
    for track in tracks:
        if not track.is_confirmed():
            continue
        track_id = track.track_id
        l, t, r, b = track.to_ltrb(orig=True)
        second_results.append({'track_id': track_id, 'bbox': [l, t, r, b]})

    for res in second_results:
        l, t, r, b = res['bbox']
        pose_res = inference_topdown(pose_model, frame, bboxes=np.array([[l, t, r, b]], dtype=np.float32))
        pred_instances = pose_res[0].pred_instances
        if pred_instances.keypoints is not None and len(pred_instances.keypoints) > 0:
            res['keypoints'] = pred_instances.keypoints[0]

    second_model_time = time.time() - start_time
    second_model_times.append(second_model_time)

    # Сопоставление детекций YOLO и второй модели
    for yolo_box, yolo_kp in zip(yolo_boxes, yolo_keypoints):
        max_iou = 0
        matched_second_res = None
        for second_res in second_results:
            second_box = second_res['bbox']
            iou = calculate_iou(yolo_box, second_box)
            if iou > max_iou:
                max_iou = iou
                matched_second_res = second_res

        if max_iou > 0.5 and 'keypoints' in matched_second_res:  # Порог IOU
            second_kp = matched_second_res['keypoints']
            if yolo_kp.shape == (17, 2) and second_kp.shape == (17, 2):
                # Разница между ключевыми точками
                diff = yolo_kp - second_kp
                mae = np.mean(np.abs(diff))
                mse = np.mean(diff ** 2)

                # Добавление метрик в список
                iou_list.append(max_iou)
                mae_list.append(mae)
                mse_list.append(mse)

    # Визуализация для YOLO
    annotated_frame_yolo = yolo_results[0].plot()
    out_yolo.write(annotated_frame_yolo)

    # Визуализация для второй модели
    frame_second = frame.copy()
    for res in second_results:
        l, t, r, b = res['bbox']
        cv2.rectangle(frame_second, (int(l), int(t)), (int(r), int(b)), (0, 255, 0), 2)
        if 'keypoints' in res:
            for kp in res['keypoints']:
                cv2.putText(frame_second, f"ID: {res['track_id']}", (int(l), int(t) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                cv2.circle(frame_second, tuple(kp.astype(int)), 5, (0, 0, 255), -1)
    out_second.write(frame_second)

# Закрытие видеофайлов и освобождение ресурсов
cap.release()
out_yolo.release()
out_second.release()


# Расчёт средних значений метрик
mean_iou = np.mean(iou_list) if iou_list else 0
mean_mae = np.mean(mae_list) if mae_list else 0
mean_mse = np.mean(mse_list) if mse_list else 0

# Среднее время обработки
mean_yolo_time = np.mean(yolo_times)
mean_second_model_time = np.mean(second_model_times)

# Вывод статистики
print(f"Путь к исходному видео: {video_path}")
print(f"Путь к сохранённому видео с результатами YOLO:")
print(f"{yolo_output_path}")
print(f"Путь к сохранённому видео с результатами второй модели:")
print(f"{second_output_path}")
print(f"Средний IOU: {mean_iou:.4f}")
print(f"Средний MAE: {mean_mae:.4f}")
print(f"Средний MSE: {mean_mse:.4f}")
print(f"Среднее время обработки кадра моделью YOLO: {mean_yolo_time:.4f} сек.")
print(f"Среднее время обработки кадра второй моделью: {mean_second_model_time:.4f} сек.")


Loads checkpoint by local backend from path: hrnet_w48_coco_256x192.pth
▶️ Обработка видео: input_video.mp4, кадры 2152-3226


  0%|          | 0/1075 [00:00<?, ?it/s]


0: 384x640 2 persons, 9.9ms
Speed: 2.0ms preprocess, 9.9ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)


  0%|          | 1/1075 [00:00<07:13,  2.48it/s]


0: 384x640 1 person, 10.9ms
Speed: 2.9ms preprocess, 10.9ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)


...


0: 384x640 1 person, 16.5ms
Speed: 2.4ms preprocess, 16.5ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)


100%|█████████▉| 1074/1075 [17:09<00:00,  2.12it/s]


0: 384x640 1 person, 18.3ms
Speed: 2.5ms preprocess, 18.3ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)


100%|██████████| 1075/1075 [17:09<00:00,  1.04it/s]

Путь к исходному видео: /content/input_video.mp4
Путь к сохранённому видео с результатами YOLO:
/content/output/output_yolo.mp4
Путь к сохранённому видео с результатами второй модели:
/content/output/output_second.mp4
Средний IOU: 0.7690
Средний MAE: 14.2874
Средний MSE: 1164.7310
Среднее время обработки кадра моделью YOLO: 0.0263 сек.
Среднее время обработки кадра второй моделью: 0.8966 сек.


In [11]:
# Распаковка
!unzip videos.zip -d /content/test_videos/

Archive:  videos.zip
  inflating: /content/test_videos/0LtLS9wROrk_E_000176_000204.mp4  
  inflating: /content/test_videos/0LtLS9wROrk_E_000217_000300.mp4  
  inflating: /content/test_videos/0LtLS9wROrk_E_000322_000417.mp4  
  inflating: /content/test_videos/0LtLS9wROrk_E_000519_000594.mp4  
  inflating: /content/test_videos/0LtLS9wROrk_E_000731_000738.mp4  
  inflating: /content/test_videos/0LtLS9wROrk_E_000773_000802.mp4  
  inflating: /content/test_videos/0LtLS9wROrk_E_000925_001024.mp4  
  inflating: /content/test_videos/0MtilFKz4cA_E_001951_001980.mp4  
  inflating: /content/test_videos/0MtilFKz4cA_E_002361_002435.mp4  


In [10]:
# @title def calculate_iou(box1, box2) / class VideoFrameCollector

# Функция для вычисления IOU между двумя боксами
def calculate_iou(box1, box2):
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])
    intersection = max(0, x2 - x1) * max(0, y2 - y1)
    area1 = (box1[2] - box1[0]) * (box1[3] - box1[1])
    area2 = (box2[2] - box2[0]) * (box2[3] - box2[1])
    union = area1 + area2 - intersection
    return intersection / union if union > 0 else 0


class VideoFrameCollector:
    def __init__(self, videos_dir, target_frames=1000, n_frames_per_video=10, max_frames_per_video=100):
        """
        Инициализация сборщика кадров видео

        :param videos_dir: Путь к директории с видеофайлами
        :param target_frames: Общее целевое количество кадров
        :param n_frames_per_video: Базовое количество кадров для каждого видео (N)
        :param max_frames_per_video: Максимальное количество кадров на одно видео
        """
        self.videos_dir = videos_dir
        self.target_frames = target_frames
        self.n_frames_per_video = n_frames_per_video
        self.max_frames_per_video = max_frames_per_video
        self.frames_list = []
        self.processed_videos = []

    def collect_frames(self, sort_by='length'):
        """
        Сбор кадров из видеофайлов

        :param sort_by: Критерий сортировки видео ('length' - по длине, 'name' - по имени)
        """
        video_files = sorted(glob.glob(f'{self.videos_dir}/*.mp4'))
        print(f"Найдено видео: {len(video_files)}")

        # Сбор информации о видео
        video_info = []
        for video_path in video_files:
            cap = cv2.VideoCapture(video_path)
            total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
            video_name = os.path.basename(video_path)
            video_info.append({
                'path': video_path,
                'name': video_name,
                'total_frames': total_frames
            })
            cap.release()

        # Сортировка видео
        if sort_by == 'length':
            video_info.sort(key=lambda x: x['total_frames'], reverse=True)
            print("Сортировка видео по длине (от самого длинного)")
        elif sort_by == 'name':
            video_info.sort(key=lambda x: x['name'])
            print("Сортировка видео по имени")
        else:
            print("Используется порядок видео без сортировки")

        collected = 0
        self.frames_list = []
        self.processed_videos = []

        # Сбор кадров
        for video in video_info:
            if collected >= self.target_frames:
                break

            # Определение количества кадров для этого видео
            n_frames = min(
                self.n_frames_per_video,
                self.max_frames_per_video,
                self.target_frames - collected
            )

            # Пропуск слишком коротких видео
            if n_frames <= 0 or video['total_frames'] < 5:
                continue

            # Выбор кадров
            frame_indices = np.linspace(0, video['total_frames']-1, n_frames, dtype=int)

            cap = cv2.VideoCapture(video['path'])
            for idx in frame_indices:
                cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
                ret, frame = cap.read()
                if ret:
                    self.frames_list.append({
                        'frame': frame,
                        'video_name': video['name'],
                        'frame_index': idx
                    })
            cap.release()

            collected += n_frames
            self.processed_videos.append({
                'video_name': video['name'],
                'frames_taken': n_frames,
                'total_frames': video['total_frames']
            })

        print(f"\nСобрано кадров: {len(self.frames_list)}")
        print("Обработанные видео:")
        for vid in self.processed_videos:
            print(f" - {vid['video_name']}: {vid['frames_taken']}/{vid['total_frames']} кадров")

    def get_frames(self):
        """Возвращает собранные кадры"""
        return self.frames_list

    def get_processed_videos_info(self):
        """Возвращает информацию об обработанных видео"""
        return self.processed_videos

In [14]:
import cv2
import glob
import numpy as np
from ultralytics import YOLO
from tqdm.notebook import tqdm
import torch
import torchvision
from mmpose.apis import init_model, inference_topdown
from deep_sort_realtime.deepsort_tracker import DeepSort
import os
import time
from sklearn.metrics import mean_squared_error, mean_absolute_error


# Устройство для вычислений
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Конфигурация
VIDEOS_DIR = '/content/test_videos'
TARGET_FRAMES = 1000
N_FRAMES_PER_VIDEO = MAX_FRAMES_PER_VIDEO = 300
V = 5

out_yolo = cv2.VideoWriter(yolo_output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (w, h))
out_second = cv2.VideoWriter(second_output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (w, h))

POSE_CFG = 'mmpose/configs/body_2d_keypoint/topdown_heatmap/coco/td-hm_hrnet-w48_8xb32-210e_coco-256x192.py'
POSE_CKPT = 'hrnet_w48_coco_256x192.pth'

# Загрузка модели YOLO
yolo_model = YOLO("yolo11n-pose.pt")

# Инициализация моделей
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
detector = torchvision.models.detection.fasterrcnn_resnet50_fpn(weights='DEFAULT').to(device)
detector.eval()
tracker = DeepSort(max_age=20, n_init=3)

# Загрузка модели pose
if not os.path.exists(POSE_CKPT):
    os.system(f'wget -q https://download.openmmlab.com/mmpose/top_down/hrnet/hrnet_w48_coco_256x192-b9e0b3ab_20200708.pth -O {POSE_CKPT}')
pose_model = init_model(POSE_CFG, POSE_CKPT, device=device)

# Скелет для визуализации
skeleton = [[0,1],[0,2],[1,3],[2,4],[0,5],[0,6],
            [5,7],[7,9],[6,8],[8,10],[5,11],[6,12],
            [11,13],[13,15],[12,14],[14,16]]


# Инициализация и сбор кадров
collector = VideoFrameCollector(
    videos_dir=VIDEOS_DIR,
    target_frames=TARGET_FRAMES,
    n_frames_per_video=N_FRAMES_PER_VIDEO,
    max_frames_per_video=MAX_FRAMES_PER_VIDEO
)

collector.collect_frames(sort_by='length')  # Сортировать видео по длине

# Получение собранных кадров
frames_data = collector.get_frames()

# Списки для хранения метрик
iou_list = []
mae_list = []
mse_list = []
yolo_times = []
second_model_times = []
# Обработка каждого кадра

print(f"▶️ Обработка видео: {os.path.basename(video_path)}, кадры {start_frame}-{end_frame-1}")
# Время обработки каждого кадра
for frame_data in tqdm(frames_data):  # Упрощенный прогресс-бар
    frame = frame_data['frame']

    # Время обработки кадра для YOLO
    start_time = time.time()
    yolo_results = yolo_model.track(frame, persist=True, tracker="bytetrack.yaml", verbose=False)
    yolo_boxes = yolo_results[0].boxes.xyxy.cpu().numpy()
    yolo_keypoints = yolo_results[0].keypoints.xy.cpu().numpy()
    yolo_time = time.time() - start_time
    yolo_times.append(yolo_time)

    # Визуализация для YOLO (ограничиваем до V = 1)
    for i in range(min(V, len(yolo_boxes))):  # Ограничиваем вывод до V объектов
        box = yolo_boxes[i]
        keypoints = yolo_keypoints[i]
        x1, y1, x2, y2 = map(int, box)
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, f"ID: {track_id}", (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        # Рисуем ключевые точки (позу)
        for kp in keypoints:
            cv2.circle(frame, tuple(kp.astype(int)), 5, (0, 0, 255), -1)

        # Рисуем скелет
        for p, q in skeleton:
            if keypoints[p][0] > 0 and keypoints[q][0] > 0:
                pt1 = tuple(keypoints[p].astype(int))
                pt2 = tuple(keypoints[q].astype(int))
                cv2.line(frame, pt1, pt2, (255, 0, 0), 2)

    out_yolo.write(frame)

        # Вторая модель: Faster R-CNN + HRNet
    start_time = time.time()
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    input_tensor = torchvision.transforms.ToTensor()(frame_rgb).to(device)
    with torch.no_grad():
        outputs = detector([input_tensor])[0]

    boxes = outputs['boxes'].cpu().numpy()
    scores = outputs['scores'].cpu().numpy()
    labels = outputs['labels'].cpu().numpy()
    mask = (scores > 0.5) & (labels == 1)
    boxes = boxes[mask]
    scores = scores[mask]  # Важно сохранить отфильтрованные scores

    detections = []
    for i, (box) in enumerate(boxes):
        x1, y1, x2, y2 = box
        w_box, h_box = x2 - x1, y2 - y1
        detections.append(([int(x1), int(y1), int(w_box), int(h_box)], scores[i], 'person'))

    tracks = tracker.update_tracks(detections, frame=frame)


    # Собираем и сортируем подтверждённые треки по confidence
    confirmed_tracks = []
    for track in tracks:
        if track.is_confirmed():
            # Получаем confidence из последнего обнаружения трека
            if hasattr(track, 'last_detection') and track.last_detection is not None:
                confidence = track.last_detection.confidence
            else:
                # Если нет информации, используем минимальный confidence
                confidence = 0.5
            confirmed_tracks.append((confidence, track))

    # Сортируем по уверенности (от высокой к низкой)
    confirmed_tracks.sort(key=lambda x: x[0], reverse=True)

    # Берём только V самых уверенных треков
    second_results = []
    for i, (confidence, track) in enumerate(confirmed_tracks[:V]):
        track_id = track.track_id
        l, t, r, b = track.to_ltrb(orig=True)
        second_results.append({'track_id': track_id, 'bbox': [l, t, r, b]})


    for res in second_results:
        l, t, r, b = res['bbox']
        pose_res = inference_topdown(pose_model, frame, bboxes=np.array([[l, t, r, b]], dtype=np.float32))
        pred_instances = pose_res[0].pred_instances
        if pred_instances.keypoints is not None and len(pred_instances.keypoints) > 0:
            res['keypoints'] = pred_instances.keypoints[0]

    second_model_time = time.time() - start_time
    second_model_times.append(second_model_time)

    # Визуализация для второй модели
    frame_second = frame.copy()
    for res in second_results[:V-1]:  # Ограничиваем отображение до V человек
        l, t, r, b = res['bbox']
        cv2.rectangle(frame_second, (int(l), int(t)), (int(r), int(b)), (0, 255, 0), 2)
        cv2.putText(frame_second, f"ID: {res['track_id']}", (int(l), int(t) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        if 'keypoints' in res:
            for kp in res['keypoints']:
                cv2.circle(frame_second, tuple(kp.astype(int)), 5, (0, 0, 255), -1)
            for p, q in skeleton:
                if res['keypoints'][p][0] > 0 and res['keypoints'][q][0] > 0:
                    pt1 = tuple(res['keypoints'][p].astype(int))
                    pt2 = tuple(res['keypoints'][q].astype(int))
                    cv2.line(frame_second, pt1, pt2, (255, 0, 0), 2)

    out_second.write(frame_second)

    # Сопоставление детекций YOLO и второй модели
    for yolo_box, yolo_kp in zip(yolo_boxes[:V], yolo_keypoints[:V]):
        max_iou = 0
        matched_second_res = None
        for second_res in second_results:
            second_box = second_res['bbox']
            iou = calculate_iou(yolo_box, second_box)
            if iou > max_iou:
                max_iou = iou
                matched_second_res = second_res

        if max_iou > 0.5 and 'keypoints' in matched_second_res:  # Порог IOU
            second_kp = matched_second_res['keypoints']
            if yolo_kp.shape == (17, 2) and second_kp.shape == (17, 2):
                # Разница между ключевыми точками
                diff = yolo_kp - second_kp
                mae = np.mean(np.abs(diff))
                mse = np.mean(diff ** 2)

                # Добавление метрик в список
                iou_list.append(max_iou)
                mae_list.append(mae)
                mse_list.append(mse)

# Закрытие видеофайлов и освобождение ресурсов
cap.release()
out_yolo.release()
out_second.release()


# Подсчёт средних метрик
average_iou = np.mean(iou_list) if iou_list else 0
average_mae = np.mean(mae_list) if mae_list else 0
average_mse = np.mean(mse_list) if mse_list else 0
average_yolo_time = np.mean(yolo_times) if yolo_times else 0
average_second_model_time = np.mean(second_model_times) if second_model_times else 0
print("____________result____________")
print(f"Средний IOU: {average_iou:.4f}")
print(f"Средний MAE: {average_mae:.4f}")
print(f"Средний MSE: {average_mse:.4f}")
print(f"Среднее время для YOLO: {average_yolo_time:.4f} секунд")
print(f"Среднее время для второй модели: {average_second_model_time:.4f} секунд")
print(f"Видео с результатами YOLO сохранено в: {yolo_output_path}")
print(f"Видео с результатами второй модели сохранено в: {second_output_path}")



Loads checkpoint by local backend from path: hrnet_w48_coco_256x192.pth
Найдено видео: 9
Сортировка видео по длине (от самого длинного)

Собрано кадров: 1000
Обработанные видео:
 - 0LtLS9wROrk_E_000925_001024.mp4: 300/2989 кадров
 - 0LtLS9wROrk_E_000322_000417.mp4: 300/2860 кадров
 - 0LtLS9wROrk_E_000217_000300.mp4: 300/2480 кадров
 - 0LtLS9wROrk_E_000519_000594.mp4: 100/2248 кадров
▶️ Обработка видео: input_video.mp4, кадры 2152-3226


  0%|          | 0/1000 [00:00<?, ?it/s]

____________result____________
Средний IOU: 0.7141
Средний MAE: 12.8429
Средний MSE: 1232.1106
Среднее время для YOLO: 0.0200 секунд
Среднее время для второй модели: 0.7013 секунд
Видео с результатами YOLO сохранено в: /content/output/output_yolo.mp4
Видео с результатами второй модели сохранено в: /content/output/output_second.mp4
